In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import os
from tqdm.notebook import tqdm
import pickle
import matplotlib.pyplot as plt

In [6]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [3]:
#Esta función devuelve una tupla con la ruta del fichero y luego nos quedamos con la 'fecha' del fichero
def file_iterator(path):
    """
    DESCRIPTION
      This function retreives all files in a specific folder and save the result in a pickle file
    ARGUMENTS
      path: Where we want to get all files
    RETURN
      A list of tuples, first element is the path and the second is the file name without any extension
    """
    
    return [(os.path.join(path,sub,file),file.split('.')[0]) for sub in os.listdir(path) 
            for file in os.listdir(os.path.join(path,sub))]

In [4]:
#Esta función crea un pickle para saber el último refresco del fichero de stock
def file_to_pickle(files, pkl_file):
    """
    DESCRIPTION
      This function saves the files tuple in a pickle. We would use it as 'latest stock files updated'. Then we
      just need to check the difference once we add a new stock file and pass those new ones to the update DataFrame
      function
    ARGUMENTS
      files: This is list of tuples, first element is the path and the second is the file name without any extension
      pkl_file: Pickle file name
    RETURN
      Nothing, it will create a pickle file
    """
    
    pkl = open(pkl_file, 'wb')
    pickle.dump(files, pkl)                     
    pkl.close()

In [5]:
#Esta función recupera el último estado del stock almanecado en un pickle
def pickle_to_file(pkl_file):
    """
    DESCRIPTION
      This function retrieves the pickle files tuple and return it
    ARGUMENTS
      pkl_file: Pickle file name
    RETURN
      A list of tuples, first element is the path and the second is the file name without any extension
    """    
    
    pkl = open(pkl_file, 'rb')     
    files = pickle.load(pkl)
    pkl.close()
    return files

In [6]:
#Comprueba que el fichero pickle existe
def pickle_exist(pkl_file):
    """
    DESCRIPTION
      This function check if the pickle file exists
    ARGUMENTS
      pkl_file: Pickle file name
    RETURN
      True or False
    """      
    
    if os.path.isfile(pkl_file):
        return True
    else:
        return False

In [7]:
#Devuelve el listado de los nuevos ficheros de stock que hay que añadir
def file_checker(path, pkl_file):
    """
    DESCRIPTION
      This function check if there is new stock files and return the new list of new files
      Good learning from here: https://www.geeksforgeeks.org/python-intersection-two-lists/
    ARGUMENTS
      path: Where we want to get all new files
      pkl_file: Pickle file name
    RETURN
      A list of tuples, first element is the path and the second is the file name without any extension
    """
    
    #https://www.geeksforgeeks.org/python-intersection-two-lists/
    #Create two sets one with the lates pickle file and the current file and get the difference
    current_files = pickle_to_file(pkl_file)
    new_files = file_iterator(path)
    #Return the difference of files
    return list(set(new_files) - set(current_files))

In [8]:
#Devuelve un DataFrame con toda la información que necesitamos
def file_extractor(files):
    """
    DESCRIPTION
      This function return a concatenated DataFrame with all desired columns
    ARGUMENTS
      files: This is list of tuples, first element is the path and the second is the file name without any extension
    RETURN
      A concatenated DataFrame with 10 columns based on the files contained in the argument
    """
    
    #A list of DataFrames, where then we will concatenated
    stock = []
    #This is the returned DataFrame
    pd_stock = pd.DataFrame()
    
    #Progression bar
    pbar = tqdm()
    pbar.reset(len(files))
    
    #Files is a Tuple made up of (path,name without extension)
    for path, name in files:
        #Read the file
        aux_stock = pd.read_excel(path, sheet_name='Daily Stock Report')        
        
        #Get the desired columns and this is difficult because the format has changed across years
        #Format 2 has this condition aux_stock.columns[0] == 'Country' else Format 1
        if aux_stock.columns[0] != 'Country':
            #First of all pick the Tablets from the old version - Condition NaN Column 2 and Tablet Column 5
            aux_stock_tablet = aux_stock[(aux_stock.iloc[:,4].str.lower() == 'tablet') 
                                         & (aux_stock.iloc[:,1].isna())].iloc[:,[1,2,3,4,5,9]]
            #Need to add Spain in one column
            aux_stock_tablet.fillna('Spain', inplace=True)

            #Secondly, get the rest of the products from Spain
            aux_stock = aux_stock[aux_stock.iloc[:,1] == 'Spain'].iloc[:,[1,2,3,4,5,9]]
            #Merge both DataFrame
            aux_stock = pd.concat([aux_stock_tablet, aux_stock], ignore_index=True)
        else:
            #There are a lot of conditions which has changed across years
            aux_stock = aux_stock[(aux_stock.iloc[:,0] == 'Shared') | (aux_stock.iloc[:,0] == 'Shared - Group II') 
                                  | (aux_stock.iloc[:,0].str.contains('ES')) 
                                  | (aux_stock.iloc[:,0] == 'Spain')].iloc[:,[0,1,2,3,4,8]]
        
            
        #We will use the file name for creating the Date, Year, Month, Day and Week Number
        year, month, day = name.split('-')
        #Important we need to keep the ISO_Year, so the year from the file sometimes is not correct
        year, week, weekday = pd.to_datetime(name).isocalendar()
        #Add the Date columns
        aux_stock['Year'] = int(year)
        aux_stock['Month'] = int(month)
        aux_stock['Day'] = int(day)
        aux_stock['Week'] = week
        if week < 10:
            aux_stock['Year-Week'] = str(year) + '-0' + str(week)
        else:
            aux_stock['Year-Week'] = str(year) + '-' + str(week)
        
        #We add this NUMPY ARRAY because the columns have different names
        stock.append(aux_stock.values)
        #Progress bar
        pbar.update()
    
    #Progress bar
    pbar.refresh()
    #Concatenated all NUMPY arrays and convert to DataFrame  
    pd_stock = pd.DataFrame(np.concatenate(stock))
    
    #New the columns name because they are unknow, just 0, 1, 2...
    columns = ['Country', 'PN', 'Product', 
               'Brand', 'Stock', 'Available', 'Year', 'Month', 'Day', 'Week', 'Year-Week']
    pd_stock.columns = columns
    #Return the final DataFrame in the desired order
    return pd_stock[['Year', 'Month', 'Day', 'Week','Year-Week', 'Country', 'PN', 'Product', 
               'Brand', 'Stock', 'Available']].sort_values(by=['Year', 'Month', 'Day'])

In [9]:
#Comprueba que el fichero Excel existe
def excel_exist(excel_file):
    """
    DESCRIPTION
      This function check if the stock Excel file exists
    ARGUMENTS
      excel_file: Stock Excel file name
    RETURN
      True or False
    """      
    
    if os.path.isfile(excel_file):
        return True
    else:
        return False

In [10]:
#Convierte el DataFrame en un fichero Excel o lo añade al último creado
def file_to_excel(stock, excel_file):
    """
    DESCRIPTION
      This function creates the stock Excel file. If could be just the full stock file or an append
    ARGUMENTS
      stock: This is the latest stock DataFrame to convert to Excel or to append to the current stock file
      excel_file: Stock Excel file name
    RETURN
      Void
    """       

    if excel_exist(excel_file):
        stock = pd.concat([stock, pd.read_excel(excel_file)], ignore_index=True)
        stock.sort_values(by=['Year', 'Month', 'Day'], inplace=True)
    stock.to_excel(excel_file, index=False)

In [11]:
#Devuelve un DataFrame de un fichero excel
def excel_to_DataFrame(excel_file):
    """
    DESCRIPTION
      This function read an Excel file and return a DataFrame
    ARGUMENTS
      excel_file: Stock Excel file name
    RETURN
      DataFrame with the stock
    """ 
    
    return pd.read_excel(excel_file)

In [23]:
#Esto deja el fichero de stock niquelado
def file_cleanup(stock):
    """
    DESCRIPTION
      This function cleanup the stock file because it has TABLET instead of Tablets and there are some NaN in Brand
      which have to rename for Phone and Option abd some NaN in Description... which has to be Tablet
    ARGUMENTS
      stock: This is the Mega DataFrame with all stock files aggreggated 
    RETURN
      File cleanup with the new scheme
    """
    
    stock.loc[(stock['Brand'].isnull()) & (stock['PN'].str.contains('PA')), 'Brand'] = 'Phone'
    stock.loc[(stock['Brand'].isnull()) & (stock['PN'].str.contains('GX')), 'Brand'] = 'Option'
    stock.loc[(stock['Product'].isnull()) & (stock['PN'].str.contains('ZA')), 'Product'] = 'Tablet'
    stock.loc[stock['Brand'] == 'TABLET', 'Brand'] = 'Tablet'
    stock.loc[stock['Brand'] == 'phone', 'Brand'] = 'Phone'
    #Return the cleaned stock file
    return stock

In [13]:
#Se queda con los duplicados de la última semana
def keep_latest_week(stock):
    """
    DESCRIPTION
      This function return the stock file with the latest week data
    ARGUMENTS
      stock: This is the Mega DataFrame with all stock files aggreggated 
    RETURN
      File with the latest week data
    """
    
    #Just two columns same week and same PartNumber/SKU as filter
    return stock[~stock.duplicated(keep = 'last', subset = ['Year-Week', 'PN'])].sort_values(by=['Year-Week'])

In [14]:
#Esta es la llamada principal que hace todo
def create_stock_file(path, pkl_file, excel_file):
    """
    DESCRIPTION
      This function create the stock Excel file from the scratch or add to the end
    ARGUMENTS
      path: Where we want to get all new files
      pkl_file: Pickle file name
      excel_file: Stock Excel file name
    RETURN
      Void
    """    
    
    #Get the new tuples of files based on pickle if empty, it creates the full list
    if not pickle_exist(pkl_file):
        files = file_iterator(path)
    else:
        files = file_checker(path, pkl_file)
    
    #If the List of Tuples is not empty, if there is files to add
    if files:
        #Create the DataFrame and create the Excel File
        stock = file_extractor(files)
        stock = file_cleanup(stock)
        stock = keep_latest_week(stock)
        file_to_excel(stock, excel_file)
    
    #Finally, we create the pickle file which is the latest stock file status
    files = file_iterator(path)
    file_to_pickle(files, pkl_file)

In [2]:
#Constants
stock_path = '/home/dsc/Repos/TFM/Stock'
stock_excel = 'stock.xlsx'
pickle_path = 'stock_files.pkl'

In [16]:
#This is the starting point. Everytime executes this program to update the stock file
create_stock_file(stock_path, pickle_path, stock_excel)

### DESDE AQUI

In [17]:
stock = excel_to_DataFrame(stock_excel)

In [3]:
stock = pd.read_excel(stock_excel)

In [7]:
stock

,Year,Month,Day,Week,Year-Week,Country,PN,Product,Brand,Stock,Available
0,2015,3,11,11,2015-11,Spain,59428120,YOGA Tablet 2 Pro–1380F 32GPT-DE,Tablet,145,145
1,2015,3,11,11,2015-11,Spain,59413057,U330Touch,Notebook,0,0
2,2015,3,11,11,2015-11,Spain,59436754,Y50-70,Notebook,4,4
3,2015,3,11,11,2015-11,Spain,59436756,Y50-70,Notebook,11,10
4,2015,3,11,11,2015-11,Spain,59440919,Y50-70,Notebook,5,5
...,...,...,...,...,...,...,...,...,...,...,...
43198,2021,4,12,15,2021-15,BE/DE/AT/CH/DK/ES/FI/FR/GB/IE/IT/NL/NO/PT/SE,GY50Z18986,MICE_BO 530MiceBlue L300,Option,146,146
43199,2021,4,12,15,2021-15,BE/DE/AT/CH/DK/ES/FI/FR/GB/IE/IT/NL/NO/PT/SE,GY50Z18984,MICE_BO 530MiceGrey L300,Option,152,148
43200,2021,4,12,15,2021-15,BE/DE/AT/CH/DK/ES/FI/FR/GB/IE/IT/NL/NO/PT/SE,GY50X88832,MICE_BO Lenovo 600 BT Silent Mouse,Option,77,75
43201,2021,4,12,15,2021-15,BE/DE/AT/CH/DK/ES/FI/FR/GB/IE/IT/NL/NO/PT/SE,GY51C96033,MICE_BO Legion M600 Mice_Stingray,Option,0,0


In [36]:
grupo = stock.groupby(['Year', 'Month', 'Day', 'Product'])['Stock'].sum().unstack().reset_index()
grupo

Product,Year,Month,Day,Lenovo 135W AC Adapter(CE-SDC),Lenovo 135W AC Adapter(CESDC),Lenovo 135W AC Adapter(UK-SDC),Lenovo 170W AC Adapter(CE-SDC),Lenovo 170W AC Adapter(CESDC),Lenovo 90W AC Adapter(CE-SDC),Lenovo 90W AC Adapter(CESDC),Lenovo 90W AC Adapter(UK-SDC),100-15,100S-11IBY,110S-11,320-15,320S-14,500-15ISK,500S-14ISK,520-15,710S-13,A540,A7-30,A740,A740,A8-50,ACCKIT_BO IPG Mouse Pad L (Brand),ACCKIT_BO IPG Mouse Pad L (Hexagon),ACCKIT_BO IPG Mouse Pad M,ACCKIT_BO Legion MousePad L,ACCKIT_BO Legion MousePad L-Grey,ACCKIT_BO Legion MousePad XXL,ACCKIT_BO Lenovo Legion Mouse Pad,ADAPTR TP 45W AC Adapter SlimTip,AUDIO_BO Cosonic Analog Headset-Grey,AUDIO_BO H200 Gaming Headset,AUDIO_BO H300 Gaming Headset,AUDIO_BO H500 Gaming Headset,AUDIO_BO H600 Headset Stingray,AUDIO_BO H600 Wireless Gaming Headset,AUDIO_BO Legion 7.1 Surround Headset,AUDIO_BO Legion Stereo Headset-ROW,AUDIO_BO Legion Stereo HeadsetROW,AUDIO_BO Lenovo 100 Headphone-Black,AUDIO_BO Lenovo 100 Headphone-White,AUDIO_BO Lenovo 100 HeadphoneBlack,AUDIO_BO Lenovo 100 HeadphoneWhite,AUDIO_BO Lenovo 100 Stereo USB HS,AUDIO_BO Lenovo 110 Stereo USB HS,AUDIO_BO Lenovo 700 BT Speaker,AUDIO_BO Lenovo VoIP Stereo Headset,...,Z70-80,e6 play,ideacentre AIO 510S-23ISU,ideacentre AIO 700-24ISH,ideacentre AIO 910-27ISH,ideacentre Y900-34ISZ,ideapad 100S-11IBY,ideapad 100S-14IBR,ideapad 520S-14,ideapad 720,ideapad 720S-14,ideapad MIIX 510-12ISK,ideapad MIIX 700-12ISK,moto G10 Power - Sakura Pearl,moto G10 Power - Aurora Grey,moto G30 - Dark Pearl,moto G30 - Pastel Sky,moto e6i - grey,moto e6i - pink,moto e7 - MISTY BLUE,moto e7 - bluebird,moto e7 - ice flow,moto e7 - water red,moto e7 plus - MISTY BLUE,moto e7 plus - twilight orange,moto e7 power - blue,moto e7 power - red,moto g 5G - grey,moto g 5G - silver,moto g 5G plus,moto g 5G plus – mystic lilac,moto g 5G plus – surfing blue,moto g pro,moto g100 - iridescent ocean,moto g100 - iridescent sky,moto g8 plus,moto g9 play – Forest Green,moto g9 play – Sapphire Blue,moto g9 play – Spring Pink,moto g9 plus - DEEP DIVE,moto g9 plus – Blush Gold,moto g9 power - jasper grey,moto g9 power - jazz blue,motorola edge,motorola edge+,motorola one,motorola one hyper,motorola one macro,motorola one zoom,pouch for motorola razr
0,2015,3,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,3,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,3,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,3,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,4,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,NaN,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [38]:
len(grupo.columns)
#JODER 1272 COLUMNAS!!!

1272

In [32]:
list(stock['Brand'].unique())

['Tablet', 'Notebook', 'Desktop', 'AIO', 'Phone', 'Option', 'Visual']

In [26]:
sap = pd.read_excel('sap.xlsx')

In [30]:
import sklearn

In [31]:
program

,program_Affiliate,program_Apps,program_Direct,program_Display,program_Email,program_Paid Social,program_SEM,program_SEO
0,0,0,1,0,0,0,0,0
1,0,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0
4,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
719,0,0,0,0,0,0,0,1
720,0,0,0,0,0,0,1,0
721,1,0,0,0,0,0,0,0
722,0,0,0,0,0,0,1,0
